# This sample was inspired by [04-context-variables-chat.ipynb](https://github.com/microsoft/semantic-kernel/blob/main/python/notebooks/04-context-variables-chat.ipynb)

## [How does Python SK compare to the C# version of Semantic Kernel?](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)
- The two SDKs are compatible and at the core they follow the same design principles.
- Some features are still available only in the C# version, and being ported.
- Refer to the [FEATURE MATRIX](https://learn.microsoft.com/en-us/semantic-kernel/get-started/supported-languages) doc to see where things stand in matching the features and functionality of the main SK branch.
- Over time there will be some features available only in the Python version, and others only in the C# version, for example adapters to external services, scientific libraries, etc.
<br/>
- Documentation
 - [Get Started with Semantic Kernel](https://github.com/microsoft/semantic-kernel/blob/main/python/README.md)

Inspired to
 - https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/KernelSyntaxExamples/Example59_OpenAIFunctionCalling.cs
 - https://github.com/microsoft/semantic-kernel/blob/main/python/samples/kernel-syntax-examples/openai_function_calling.py

In [1]:
# create the kernel

import semantic_kernel as sk
kernel = sk.Kernel()

In [2]:
# create the SK "chat completion connector" to the Azure OpenAI service

from dotenv import load_dotenv
load_dotenv("credentials_my.env")

import semantic_kernel.connectors.ai.open_ai as sk_oai

chat_connector = sk_oai.AzureChatCompletion(
    api_key=os.environ['AZURE_OPENAI_API_KEY'],
    api_version=os.environ['AZURE_OPENAI_API_VERSION'],
    deployment_name=os.environ['GPT35TURBO-0613-4k'], # ['GPT4-1106-128k'],
    endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
)
print(f"This is my AzureChatCompletion connector:\n{chat_connector}")

This is my AzureChatCompletion connector:
ai_model_id='gpt35turbo-0613-4k' client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7f12cfbbe7b0> ai_model_type=<OpenAIModelTypes.CHAT: 'chat'> prompt_tokens=0 completion_tokens=0 total_tokens=0


In [3]:
# add the openAI completion connector to the kernel, choosing a name to identify it within the kernel

kernel.add_chat_service(
    "mauromi_chatgpt", # unique name to be registered with the kernel
    chat_connector
)
print(f"Here are all chat services registered with this kernel: {kernel.all_chat_services()}")

Here are all chat services registered with this kernel: ['mauromi_chatgpt']


In [4]:
# Let's define a prompt outlining a dialogue chat bot

sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

In [5]:
# In-Line registration of our semantic function

chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.5,
)

In [6]:
# Initialize our context

context = kernel.create_new_context()
context.variables["user_input"] = "Hi, I'm looking for book suggestions"
context.variables["history"] = "."

In [7]:
# Chat with the bot

bot_answer = chat_function.invoke(context=context)
print(bot_answer)

Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?


In [8]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])

.
User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?



In [9]:
def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = chat_function.invoke(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [10]:
chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: I would recommend "The Greeks: A Portrait of Self and Others" by Paul Cartledge. It provides a comprehensive overview of ancient Greek history, culture, and philosophy. Another great book is "The History of Greece" by Thucydides, which offers a firsthand account of the Peloponnesian War and the political events of ancient Greece.


In [11]:
chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: "The Greeks: A Portrait of Self and Others" by Paul Cartledge explores the history, culture, and philosophy of ancient Greece. It delves into the achievements and contributions of the ancient Greeks, including their political systems, literature, art, and philosophy. The book also examines the interactions between the Greeks and other civilizations, such as the Persians and the Romans. Overall, it provides a comprehensive and engaging look at the ancient Greek civilization.


In [12]:
chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read "The Greeks: A Portrait of Self and Others" by Paul Cartledge, you will learn about various aspects of Greek history. Some of the topics covered include the rise of the Greek city-states, the development of democracy in Athens, the Persian Wars, the Peloponnesian War, the conquests of Alexander the Great, and the Hellenistic period. You will also learn about the major figures in Greek history, such as Pericles, Socrates, Plato, and Aristotle. Additionally, the book explores the cultural and intellectual achievements of the Greeks, including their contributions to literature, philosophy, art, and architecture.


In [13]:
chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are some more books you could consider reading about Greek history:

1. "The Peloponnesian War" by Thucydides - This book provides a detailed account of the Peloponnesian War, a conflict between Athens and Sparta that shaped the course of Greek history.

2. "The Histories" by Herodotus - Herodotus is often referred to as the "Father of History," and his work provides a comprehensive account of the Persian Wars and other events in ancient Greece.

3. "The Rise and Fall of Athens: Nine Greek Lives" by Plutarch - This book offers biographies of nine influential figures in ancient Athens, including Pericles, Themistocles, and Alcibiades.

4. "The Classical World: An Epic History from Homer to Hadrian" by Robin Lane Fox - This book provides a broad overview of ancient Greek and Roman history, including the major events, figures, and cultural developments.

5. "The Greek Way" by Edith Hamilton - This

In [14]:
chat("Tell me something more about the second book")

User: Tell me something more about the second book
ChatBot: "The Histories" by Herodotus is a seminal work in the field of history. It is often considered the first true historical account and provides a detailed narrative of the Persian Wars, which were a series of conflicts between the Greek city-states and the Persian Empire in the 5th century BCE. Herodotus traveled extensively and collected information from various sources, including eyewitness accounts and oral traditions, to compile his work. 

"The Histories" not only chronicles the military events of the Persian Wars but also delves into the cultural, social, and political aspects of the time. Herodotus explores the motivations and actions of both the Greeks and the Persians, offering insights into their respective cultures and worldviews. The book also includes fascinating anecdotes and stories, such as the Battle of Thermopylae and the Battle of Marathon, which have become iconic in Greek history.

Overall, "The Histories" i

In [15]:
print(context["history"])

.
User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: I would recommend "The Greeks: A Portrait of Self and Others" by Paul Cartledge. It provides a comprehensive overview of ancient Greek history, culture, and philosophy. Another great book is "The History of Greece" by Thucydides, which offers a firsthand account of the Peloponnesian War and the political events of ancient Greece.

User: that sounds interesting, what is it about?
ChatBot: "The Greeks: A Portrait of Self and Others" by Paul Cartledge explores the history, culture, and philosophy of ancient Greece. It delves into the achievements and contributions of the ancient Greeks, including their political systems, literature, art, and philosophy. The book also examines the interactions between the Greeks and other civili

Bad pipe message: %s [b'\xb9\x1c\xe3i7<K\xfb\xdb\x9b};\xf15']
Bad pipe message: %s [b'\xafu \xf6\xca\x17i\x88t\xb2n\xfcR\xb2\xa4\xbeQ\xa5\x8f\xe7\xce\t\xff\xed\xb6}bt\xce6\xb8GQXW\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x88+\xba\xd5sNH\x8b\xb1\xd0']
Bad pipe message: %s [b'#Z\xd4\xd0+u\xdbI\xf7\xeb\x88_\xa0\xe06u\xc5: \xc1I\xb1\x92\xd0t\xb6\xe3\x88\x9f\x8f\xd6C\x05\x8bfE\xafg/\xbd7\xbe\x87\xd4}\xc7\x1e$\xf2AK\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\